In [1]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

conn = sqlite3.connect('example.db')
def rsq(sql):
    df = pd.read_sql(sql, conn)
    return df

In [2]:
rsq("""SELECT 
    -- Select country ID, date, home, and away goals from match
    main.country_id,
    main.date,
    main.home_goal,
    main.away_goal
FROM match AS main
WHERE 
    -- Filter the main query by the subquery
    (home_goal + away_goal) > 
        (SELECT AVG((sub.home_goal + sub.away_goal) * 3)
         FROM match AS sub
         -- Join the main query to the subquery in WHERE
         WHERE main.country_id = sub.country_id);""")

,country_id,date,home_goal,away_goal
0,7809,2013-01-18,5,4.0
1,7809,2015-02-14,4,5.0
2,7809,2014-10-25,4,5.0
3,10257,2013-03-30,3,5.0
4,7809,2013-03-30,9,2.0
5,21518,2013-10-30,7,3.0
6,21518,2015-04-05,9,1.0
7,21518,2015-05-23,7,3.0


In [3]:
rsq("""SELECT 
    -- Select country ID, date, home, and away goals from match
    main.country_id,
    main.date,
    main.home_goal,
    main.away_goal
FROM match AS main
WHERE 
    -- Filter for matches with the highest number of goals scored
    (home_goal + away_goal) =
        (SELECT MAX(sub.home_goal + sub.away_goal)
         FROM match AS sub
         WHERE main.country_id = sub.country_id
               AND main.season = sub.season);""")

,country_id,date,home_goal,away_goal
0,7809,2012-03-10,7,1.0
1,7809,2012-03-30,4,4.0
2,7809,2013-11-09,5,3.0
3,7809,2013-11-30,4,4.0
4,7809,2014-03-02,6,2.0
5,7809,2015-02-14,4,5.0
6,7809,2014-10-25,4,5.0
7,10257,2011-12-21,6,1.0
8,10257,2011-10-23,3,4.0
9,10257,2013-03-30,3,5.0


In [4]:
rsq("""SELECT 
    -- Select country ID, date, home, and away goals from match
    main.country_id,
    main.date,
    main.home_goal,
    main.away_goal
FROM match AS main
WHERE 
    -- Filter for matches with the highest number of goals scored
    (home_goal + away_goal) =
        (SELECT MAX(sub.home_goal + sub.away_goal)
         FROM match AS sub
         WHERE main.country_id = sub.country_id
               AND main.season = sub.season);""")

,country_id,date,home_goal,away_goal
0,7809,2012-03-10,7,1.0
1,7809,2012-03-30,4,4.0
2,7809,2013-11-09,5,3.0
3,7809,2013-11-30,4,4.0
4,7809,2014-03-02,6,2.0
5,7809,2015-02-14,4,5.0
6,7809,2014-10-25,4,5.0
7,10257,2011-12-21,6,1.0
8,10257,2011-10-23,3,4.0
9,10257,2013-03-30,3,5.0


In [8]:
rsq("""SELECT 
	-- Select the season and max goals scored in a match
	season,
    MAX(home_goal + away_goal) AS max_goals,
    -- Select the overall max goals scored in a match
   (SELECT MAX(home_goal + away_goal) FROM match) AS overall_max_goals,
    -- Select the max number of goals scored in any match in July
   (SELECT MAX(home_goal + away_goal) 
        FROM match
        WHERE id IN (
              SELECT id FROM match WHERE strftime('%m', date) = '07')) AS july_max_goals
--#             SELECT id FROM match WHERE EXTRACT(MONTH FROM date) = 07)) AS july_max_goals
FROM match
GROUP BY season;""")

,season,max_goals,overall_max_goals,july_max_goals
0,2011/2012,10.0,11.0,None
1,2012/2013,11.0,11.0,None
2,2013/2014,10.0,11.0,None
3,2014/2015,10.0,11.0,None


In [10]:
rsq("""-- Select matches where a team scored 5+ goals
SELECT
    country_id,
    season,
    id
FROM match
WHERE home_goal >= 5 OR away_goal >= 5;""")

,country_id,season,id
0,7809,2011/2012,8736
1,7809,2011/2012,8754
2,7809,2011/2012,8771
3,7809,2011/2012,8801
4,7809,2011/2012,8810
...,...,...,...
74,7809,2012/2013,9211
75,13274,2011/2012,14224
76,21518,2013/2014,23444
77,21518,2014/2015,24016


In [12]:
rsq("""-- Count match ids
SELECT
    country_id,
    season,
    COUNT(id) AS matches
-- Set up and alias the subquery
FROM (
    SELECT
        country_id,
        season,
        id
    FROM match
    WHERE home_goal >= 5 OR away_goal >= 5) 
    AS subquery
-- Group by country_id and season
GROUP BY country_id, season;""")

,country_id,season,matches
0,1729,2011/2012,1
1,1729,2012/2013,2
2,7809,2011/2012,11
3,7809,2012/2013,9
4,7809,2013/2014,12
5,7809,2014/2015,10
6,10257,2011/2012,2
7,10257,2012/2013,5
8,13274,2011/2012,1
9,21518,2011/2012,23


In [13]:
rsq("""SELECT
    c.name AS country,
    -- Calculate the average matches per season
    AVG(outer_s.matches) AS avg_seasonal_high_scores
FROM country AS c
-- Left join outer_s to country
LEFT JOIN (
  SELECT country_id, season,
         COUNT(id) AS matches
  FROM (
    SELECT country_id, season, id
    FROM match
    WHERE home_goal >= 5 OR away_goal >= 5) AS inner_s
  -- Close parentheses and alias the subquery
  GROUP BY country_id, season) AS outer_s
ON c.id = outer_s.country_id
GROUP BY country;""")

,country,avg_seasonal_high_scores
0,Belgium,NaN
1,England,1.500000
2,France,NaN
3,Germany,10.500000
4,Italy,3.500000
5,Netherlands,1.000000
6,Poland,NaN
7,Portugal,NaN
8,Scotland,NaN
9,Spain,8.666667


In [14]:
rsq('''-- Set up your CTE
WITH match_list AS (
    SELECT 
        country_id, 
        id
    FROM match
    WHERE (home_goal + away_goal) >= 10)
-- Select league and count of matches from the CTE
SELECT
    l.name AS league,
    COUNT(match_list.id) AS matches
FROM league AS l
-- Join the CTE to the league table
LEFT JOIN match_list 
ON l.id = match_list.country_id
GROUP BY l.name;''')

,league,matches
0,Belgium Jupiler League,0
1,England Premier League,3
2,France Ligue 1,0
3,Germany 1. Bundesliga,1
4,Italy Serie A,0
5,Netherlands Eredivisie,1
6,Poland Ekstraklasa,0
7,Portugal Liga ZON Sagres,0
8,Scotland Premier League,0
9,Spain LIGA BBVA,3


In [15]:
rsq("""-- Set up your CTE
WITH match_list AS (
  -- Select the league, date, home, and away goals
    SELECT 
        l.name AS league, 
        m.date, 
        m.home_goal, 
        m.away_goal,
       (m.home_goal + m.away_goal) AS total_goals
    FROM match AS m
    LEFT JOIN league as l ON m.country_id = l.id)
-- Select the league, date, home, and away goals from the CTE
SELECT league, date, home_goal, away_goal
FROM match_list
-- Filter by total goals
WHERE total_goals >= 10;""")

,league,date,home_goal,away_goal
0,England Premier League,2011-08-28,8,2.0
1,England Premier League,2012-12-29,7,3.0
2,England Premier League,2013-05-19,5,5.0
3,Germany 1. Bundesliga,2013-03-30,9,2.0
4,Netherlands Eredivisie,2011-11-06,6,4.0
5,Spain LIGA BBVA,2013-10-30,7,3.0
6,Spain LIGA BBVA,2015-04-05,9,1.0
7,Spain LIGA BBVA,2015-05-23,7,3.0


In [16]:
rsq("""SELECT 
        country_id, 
       (home_goal + away_goal) AS goals
    FROM match
    -- Create a list of match IDs to filter data in the CTE
    WHERE id IN (
       SELECT id
       FROM match
       WHERE season = '2013/2014' AND cast(cast(strftime('%m', date) as int) as int) = 08)""")

,country_id,goals
0,7809,4.0
1,7809,5.0
2,7809,4.0
3,7809,6.0
4,7809,4.0
5,7809,2.0
6,7809,4.0
7,7809,1.0
8,7809,7.0
9,7809,3.0


In [20]:
rsq("""-- Set up your CTE
WITH match_list AS (
    SELECT 
  		country_id, 
  	   (home_goal + away_goal) AS goals
    FROM match
    -- Create a list of match IDs to filter data in the CTE
    WHERE id IN (
       SELECT id
       FROM match
       WHERE season = '2013/2014' AND EXTRACT(MONTH FROM date) = 08)) 
-- Select the league name and average of goals in the CTE
SELECT
    l.name,
    AVG(match_list.goals)
FROM league AS l
-- Join the CTE onto the league table
LEFT JOIN match_list ON l.id = match_list.country_id
GROUP BY l.name;""")

DatabaseError: Execution failed on sql '-- Set up your CTE
WITH match_list AS (
    SELECT 
  		country_id, 
  	   (home_goal + away_goal) AS goals
    FROM match
    -- Create a list of match IDs to filter data in the CTE
    WHERE id IN (
       SELECT id
       FROM match
       WHERE season = '2013/2014' AND EXTRACT(MONTH FROM date) = 08)) as x
-- Select the league name and average of goals in the CTE
SELECT
    l.name,
    AVG(match_list.goals)
FROM league AS l
-- Join the CTE onto the league table
LEFT JOIN match_list ON l.id = match_list.country_id
GROUP BY l.name;': near "FROM": syntax error

In [21]:
rsq("""SELECT 
	m.id, 
    t.team_long_name AS hometeam
-- Left join team to match
FROM match AS m
inner join team as t
ON m.hometeam_id = team_api_id;""")

,id,hometeam
0,8727,Borussia Dortmund
1,8728,SV Werder Bremen
2,8728,SV Werder Bremen
3,8728,SV Werder Bremen
4,8729,Hannover 96
...,...,...
3514,14224,FC Utrecht
3515,23444,Real Madrid CF
3516,24016,Real Madrid CF
3517,24114,Real Madrid CF


In [23]:
rsq("""SELECT
    m.date,
    -- Get the home and away team names
    hometeam,
    awayteam,
    m.home_goal,
    m.away_goal
FROM match AS m

-- Join the home subquery to the match table
LEFT JOIN (
  SELECT match.id, team.team_long_name AS hometeam
  FROM match
  LEFT JOIN team
  ON match.hometeam_id = team.team_api_id) AS home
ON home.id = m.id

-- Join the away subquery to the match table
LEFT JOIN (
  SELECT match.id, team.team_long_name AS awayteam
  FROM match
  LEFT JOIN team
  -- Get the away team ID in the subquery
  ON match.awayteam_id = team.team_api_id) AS away
ON away.id = m.id;""")

,date,hometeam,awayteam,home_goal,away_goal
0,2011-08-05,Borussia Dortmund,Hamburger SV,3,1.0
1,2011-08-06,SV Werder Bremen,1. FC Kaiserslautern,2,0.0
2,2011-08-06,SV Werder Bremen,1. FC Kaiserslautern,2,0.0
3,2011-08-06,SV Werder Bremen,1. FC Kaiserslautern,2,0.0
4,2011-08-06,SV Werder Bremen,1. FC Kaiserslautern,2,0.0
...,...,...,...,...,...
8446,2011-11-06,FC Utrecht,Ajax,6,4.0
8447,2013-10-30,Real Madrid CF,Sevilla FC,7,3.0
8448,2015-04-05,Real Madrid CF,Granada CF,9,1.0
8449,2015-05-23,Real Madrid CF,Getafe CF,7,3.0


In [25]:
rsq("""SELECT
    m.date,
   (SELECT team_long_name
    FROM team AS t
    -- Connect the team to the match table
    WHERE t.team_api_id = m.hometeam_id) AS hometeam
FROM match AS m;""")

,date,hometeam
0,2011-08-05,Borussia Dortmund
1,2011-08-06,SV Werder Bremen
2,2011-08-06,Hannover 96
3,2011-08-07,1. FSV Mainz 05
4,2011-08-06,VfB Stuttgart
...,...,...
1608,2011-11-06,FC Utrecht
1609,2013-10-30,Real Madrid CF
1610,2015-04-05,Real Madrid CF
1611,2015-05-23,Real Madrid CF


In [27]:
rsq("""SELECT
    m.date,
   (SELECT team_long_name
    FROM team AS t
    WHERE t.team_api_id = m.hometeam_id) AS hometeam,
    -- Connect the team to the match table
   (SELECT team_long_name
    FROM team AS t
    WHERE t.team_api_id = m.awayteam_id) AS awayteam,
   -- Select home and away goals
    m.home_goal,
    m.away_goal
FROM match AS m;""")

,date,hometeam,awayteam,home_goal,away_goal
0,2011-08-05,Borussia Dortmund,Hamburger SV,3,1.0
1,2011-08-06,SV Werder Bremen,1. FC Kaiserslautern,2,0.0
2,2011-08-06,Hannover 96,TSG 1899 Hoffenheim,2,1.0
3,2011-08-07,1. FSV Mainz 05,Bayer 04 Leverkusen,2,0.0
4,2011-08-06,VfB Stuttgart,FC Schalke 04,3,0.0
...,...,...,...,...,...
1608,2011-11-06,FC Utrecht,Ajax,6,4.0
1609,2013-10-30,Real Madrid CF,Sevilla FC,7,3.0
1610,2015-04-05,Real Madrid CF,Granada CF,9,1.0
1611,2015-05-23,Real Madrid CF,Getafe CF,7,3.0


In [29]:
rsq("""SELECT 
    -- Select match id and team long name
    m.id, 
    t.team_long_name AS hometeam
FROM match AS m
-- Join team to match using team_api_id and hometeam_id
LEFT JOIN team AS t 
ON m.hometeam_id = t.team_api_id;""")

,id,hometeam
0,8727,Borussia Dortmund
1,8728,SV Werder Bremen
2,8728,SV Werder Bremen
3,8728,SV Werder Bremen
4,8729,Hannover 96
...,...,...
3514,14224,FC Utrecht
3515,23444,Real Madrid CF
3516,24016,Real Madrid CF
3517,24114,Real Madrid CF


In [30]:
rsq("""-- Declare the home CTE
WITH home AS (
  SELECT m.id, t.team_long_name AS hometeam
  FROM match AS m
  LEFT JOIN team AS t 
  ON m.hometeam_id = t.team_api_id)
-- Select everything from home
SELECT *
FROM home;""")

,id,hometeam
0,8727,Borussia Dortmund
1,8728,SV Werder Bremen
2,8728,SV Werder Bremen
3,8728,SV Werder Bremen
4,8729,Hannover 96
...,...,...
3514,14224,FC Utrecht
3515,23444,Real Madrid CF
3516,24016,Real Madrid CF
3517,24114,Real Madrid CF


In [31]:
rsq("""WITH home AS (
  SELECT m.id, m.date, 
         t.team_long_name AS hometeam, m.home_goal
  FROM match AS m
  LEFT JOIN team AS t 
  ON m.hometeam_id = t.team_api_id),
-- Declare and set up the away CTE
away AS (
  SELECT m.id, m.date, 
         t.team_long_name AS awayteam, m.away_goal
  FROM match AS m
  LEFT JOIN team AS t 
  ON m.awayteam_id = t.team_api_id)
-- Select date, home_goal, and away_goal
SELECT 
    home.date,
    home.hometeam,
    away.awayteam,
    home.home_goal,
    away.away_goal
-- Join away and home on the id column
FROM home
INNER JOIN away
ON home.id = away.id;""")

,date,hometeam,awayteam,home_goal,away_goal
0,2011-08-05,Borussia Dortmund,Hamburger SV,3,1.0
1,2011-08-06,SV Werder Bremen,1. FC Kaiserslautern,2,0.0
2,2011-08-06,SV Werder Bremen,1. FC Kaiserslautern,2,0.0
3,2011-08-06,SV Werder Bremen,1. FC Kaiserslautern,2,0.0
4,2011-08-06,SV Werder Bremen,1. FC Kaiserslautern,2,0.0
...,...,...,...,...,...
8446,2011-11-06,FC Utrecht,Ajax,6,4.0
8447,2013-10-30,Real Madrid CF,Sevilla FC,7,3.0
8448,2015-04-05,Real Madrid CF,Granada CF,9,1.0
8449,2015-05-23,Real Madrid CF,Getafe CF,7,3.0
